# 実践的データ分析講座 ３日目：ハンズオン

* １日目の座学でざっと説明した分析を、データを使って実際に行っていきます
* データは [Kaggle](https://www.kaggle.com/) に収録されている [Medical Cost Personal Datasets](https://www.kaggle.com/mirichoi0218/insurance) を使用します

In [ ]:
# おまじない
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *
import seaborn as sns
sns.set(font_scale=1.5)
import numpy as np
import pandas as pd

# 前処理

最も重要かつ大変だけど、地味で辛い作業

## データの読み込み

* CSV データを読み込む
    * 先頭行にカラムの名前が付いた、ヘッダ付き CSV
    * pandas パッケージを用いて、CSV を読み込むと共に内容を DataFrame 形式に変換し格納

In [ ]:
# CSVファイルの読み込み
data = pd.read_csv('insurance.csv', header=0)
# 先頭5行を表示
data.head(n=5)

## データの説明

ある国の住人が、医療費としていくら支払ったかのデータ

* `age`: 年齢
* `sex`: 性別
* `bmi`: ボディマス指数（Body Mass Index: 体重(kg)÷(身長(m))${}^2$）
* `children`: 養育している子供の数
* `smoker`: 喫煙しているか否か
* `region`: 住んでいる地域
* `charges`: 支払った医療費の額

In [ ]:
# データの数を調べる
data.shape

## 欠損値

In [ ]:
# 欠損しているデータ数を調べる
data.isnull().sum().sum()

In [ ]:
# データ除去前の個数確認
data.shape

In [ ]:
# データが欠損している行（データ点）を除去する
data.dropna(axis='index', how='any', inplace=True)
# データ除去後の個数確認
data.shape

## データ重複

In [ ]:
# データが重複している数を調べる
data.duplicated().sum()

In [ ]:
# データ除去前の個数確認
data.shape

In [ ]:
# 重複している行（データ点）を除去する
data.drop_duplicates(keep='first', ignore_index=True, inplace=True)
# データ除去後の個数確認
data.shape

## 分布

データを見る限り、それぞれの項目は以下のデータ種別でありそう。

* `age`: 整数の連続変数
* `sex`: 文字列のカテゴリ変数
* `bmi`: 実数の連続変数
* `children`: 整数の連続変数
* `smoker`: 文字列の
* `region`: 文字列のカテゴリ変数
* `charges`: 実数の連続変数

これらがどのような分布を示すか調べる。

In [ ]:
# `age` の分布
data['age'].plot(kind='hist', bins=int(np.sqrt(data.shape[0])))
plt.xlabel('age')
plt.title('distribution of age')

In [ ]:
from collections import Counter

# `sex` の分布
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data['sex'])).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# `bmi` の分布
data['bmi'].plot(kind='hist', bins=int(np.sqrt(data.shape[0])))
plt.xlabel('bmi')
plt.title('distribution of BMI')

In [ ]:
# `children` の分布（カテゴリ変数として）
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data['children'])).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# `children` の分布（連続変数として）
data['children'].plot(kind='hist', bins=int(max(data['children'] + 1)))
plt.xlabel('children')
plt.title('distribution of children')

In [ ]:
# `smoker` の分布
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data['smoker'])).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# `region` の分布
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data['region'])).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# `charges` の分布
data['charges'].plot(kind='hist', bins=int(np.sqrt(data.shape[0])))
plt.xlabel('charges')
plt.title('distribution of charges')

## カテゴリ変数のダミー変数化

* 対象となるカラム（カテゴリ変数）
    * `sex`
    * `smoker`
    * `region`

In [ ]:
# ダミー変数化の説明として、`region` を単純にダミー変数化し、比較する
pd.merge(
    data.loc[:, ['region']],
    pd.get_dummies(data['region'], prefix='region_is'),
    left_index=True, right_index=True
).head(n=5)

In [ ]:
# このままだとマルチコを引き起こすので、適当にひとつ除去する
pd.merge(
    data.loc[:, ['region']],
    pd.get_dummies(data['region'], prefix='region_is', drop_first=True),
    left_index=True, right_index=True
).head(n=5)

In [ ]:
# すべてのカテゴリ変数をダミー変数化し、元のデータにくっつける
data = pd.concat([
    data,
    pd.get_dummies(data['sex'], prefix='sex_is', drop_first=True),
    pd.get_dummies(data['smoker'], prefix='smoker_is', drop_first=True),
    pd.get_dummies(data['region'], prefix='region_is', drop_first=True),
], axis='columns')
# 元のカテゴリ変数を除去
data.drop(['sex', 'smoker', 'region'], axis='columns', inplace=True)
# 先頭５行を表示
data.head()

In [ ]:
# データの個数確認
data.shape

## データの統計値

In [ ]:
### describe() でデータの統計値を簡単に確認できる
data.describe().round(4)

## 異常値

* 異常値かどうかは、その値から単純に判定できない
* 分布やデータの意味を理解した上で判定しなければならない

In [ ]:
# すべてのカラムの箱ひげ図を並べて表示するため、簡易的に正規化する
data_normed = (data - data.mean(axis='index')) / data.std(axis='index')

In [ ]:
# 箱ひげ図を表示する
data_normed.plot(kind='box')
plt.xticks(rotation=90)
plt.title('box plot')

異常値をどのように判断するか

* ダミー変数は（箱ひげ図で外れ値となっても）異常値として扱わない
    * データが不均衡（imbalance）の場合は、簡単に異常値と判定されてしまう
* `bmi` の外れ値
    * BMIが大きい人（極度の肥満）は、健康に問題を抱えている可能性が高い
    * ゆえに医療費が高額になると予想される
    * 箱ひげ図としては外れ値だが、重要な意味を持つデータである
    * よって、異常値としては扱わない（除外しない）
* `children` の外れ値
    * 扶養する子供が多い人は、医療費を多く支払う可能性がある
    * 指数分布のような分布をしており、値が多い場合でも異常値とは見做せない
    * よって、異常値としては扱わない（除外しない）
* `charges` の外れ値
    * 医療費が極端に高額な人は、何か問題を抱えていると考えられ、重要なデータである
    * べき分布のような分布をしており、値が多い場合でも異常値とは見做せない
    * よって、異常値としては扱わない（除外しない）
* 以上から、今回のデータでは、異常値の除外処理は行わない

## 相関

In [ ]:
# すべてのカラムの分布と相関を一度に見る簡易的な方法
# カラム数が多い場合はお薦めできない
sns.pairplot(data)

In [ ]:
# 相関係数の絶対値をヒートマップで表示する
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.heatmap(data.corr().abs(), cmap='jet', annot=True, fmt='0.2f', ax=ax)

# モデル作成と評価（分類）

* 目的変数として `smoker_is_yes` を設定する
    * 各種データから、その人が喫煙者かどうかを予測する

## 目的変数・特徴量の分離

In [ ]:
# 目的変数・特徴量の分離
data_classify_y = data['smoker_is_yes']
data_classify_X = data.drop('smoker_is_yes', axis='columns')
# データ数の確認
data_classify_X.shape, data_classify_y.shape

## 学習用・検証用データの分離

In [ ]:
from sklearn.model_selection import train_test_split

# 学習用・検証用データの分離
# `smoker_is_yes` は imbalance なので、stratify を指定する
(
    data_classify_train_X, data_classify_test_X,
    data_classify_train_y, data_classify_test_y
) = train_test_split(
    data_classify_X, data_classify_y,
    stratify=data_classify_y,
    test_size=0.3, random_state=14
)

## 重要なポイント：再現性の確保

* scikit-learn パッケージ(sklearn)で `random_state` バラメータがある関数は、必ずこれを指定する
    * 乱数を用いるアルゴリズム
    * `random_state` を付けないと、関数を実行するたびに違う結果になる
    * 良い精度のモデルが作成できたとしても、それを再現することが出来ない

In [ ]:
# データ数の確認（学習用データ）
data_classify_train_X.shape, data_classify_train_y.shape

In [ ]:
# データ数の確認（検証用データ）
data_classify_test_X.shape, data_classify_test_y.shape

In [ ]:
# 目的変数の分布の確認（学習用データ）
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data_classify_train_y)).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# データ数に対する喫煙者の割合（学習用データ）
round(
    Counter(data_classify_train_y)[1] / data_classify_train_y.shape[0], 3
)

In [ ]:
# 目的変数の分布の確認（検証用データ）
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data_classify_test_y)).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# データ数に対する喫煙者の割合（検証用データ）
round(
    Counter(data_classify_test_y)[1] / data_classify_test_y.shape[0], 3
)

## 特徴量エンジニアリング

* 残念ながら、このデータへの知見は持ち合わせてないので、知見による特徴エンジニアリングは出来ない
* 対数化
    * `charges` の分布がべき分布に近いので、`charges` を対数化する
* 離散化
    * `charges` の分布には３つの山があるように見えるので、`charges` を３つにクラスタリングする
* 本来ならば、同じカラムに２種類の特徴量エンジニアリングはやらない
    * マルチコになるだけだから
    * 今回は例として無理に行う

In [ ]:
# `charges` の対数化
data_classify_train_X = data_classify_train_X.assign(
    log_charges=np.log(data_classify_train_X['charges'] + 1)
)

In [ ]:
# クラスタリングとしては k-Means を用いる（1次元 k-Means）
# k-Means は距離を用いるため、正規化が必要
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 正規化
ss_kmeans = StandardScaler()
data_clustering_train_X = ss_kmeans.fit_transform(
    data_classify_train_X.loc[:, ['charges']]
)
# k-Meansの学習
km_charges = KMeans(
    n_clusters=3, init='k-means++', n_init=10, max_iter=300, random_state=14
).fit(
    data_clustering_train_X
)
# クラスタ番号は適当に付くので、中心の値が小さい順に振り直す
# そのための辞書を作成
km_cluster_map = {
    c: f'cluster{i}' for i, c in enumerate(
        km_charges.cluster_centers_.ravel().argsort()
    )
}
# k-Meansの予測結果を振り直しつつ格納
data_classify_train_X = data_classify_train_X.assign(
    charge_cluster=[
        km_cluster_map[x] for x in km_charges.predict(data_clustering_train_X)
    ]
)
# k-Meansの結果を可視化
sns.displot(
    data=data_classify_train_X, x='charges', palette='tab10',
    hue='charge_cluster', hue_order=['cluster0', 'cluster1', 'cluster2']
)
plt.title('clustering (discretize) charges')

In [ ]:
# k-Meansの予測結果をダミー変数化
data_classify_train_X = pd.concat([
    data_classify_train_X,
    pd.get_dummies(
        data_classify_train_X['charge_cluster'],
        prefix='charge_is', drop_first=True
    ),
], axis='columns')
# `charges` と `charge_cluster` を除去
data_classify_train_X.drop(
    ['charges', 'charge_cluster'], axis='columns', inplace=True
)
# 先頭５行を表示
data_classify_train_X.head(n=5)

In [ ]:
# 検証用データにも同じことをする
# （学習は一切行わず、予測のみ）

# 対数化
data_classify_test_X = data_classify_test_X.assign(
    log_charges=np.log(data_classify_test_X['charges'] + 1)
)
# k-Meansのための正規化
data_clustering_test_X = ss_kmeans.transform(
    data_classify_test_X.loc[:, ['charges']]
)
# k-Meansの予測結果を格納
data_classify_test_X = data_classify_test_X.assign(
    charge_cluster=[
        km_cluster_map[x] for x in km_charges.predict(data_clustering_test_X)
    ]
)
# k-Meansの予測結果をダミー変数化
data_classify_test_X = pd.concat([
    data_classify_test_X,
    pd.get_dummies(
        data_classify_test_X['charge_cluster'],
        prefix='charge_is', drop_first=True
    ),
], axis='columns')
# `charges` と `charge_cluster` を除去
data_classify_test_X.drop(
    ['charges', 'charge_cluster'], axis='columns', inplace=True
)
# 先頭５行を表示
data_classify_test_X.head(n=5)

In [ ]:
# データ数の確認
data_classify_train_X.shape, data_classify_test_X.shape

In [ ]:
# 特徴量を追加したため、相関を再確認する
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.heatmap(
    data_classify_train_X.corr().abs(),
    cmap='jet', annot=True, fmt='0.2f', ax=ax
)

## 正規化

* Support Vector Machine のために、データを正規化する
    * Random Forest は正規化したデータは使わない

In [ ]:
# 正規化
ss_classify = StandardScaler().fit(
    data_classify_train_X
)
# 学習用データ
data_classify_train_X_normed = ss_classify.transform(
    data_classify_train_X
)
# 検証用データ
data_classify_test_X_normed = ss_classify.transform(
    data_classify_test_X
)

## 次元圧縮

今回は次元圧縮したデータは使わないが、例として行う

In [ ]:
from sklearn.decomposition import PCA

# 圧縮しても十分に意味が残る累積寄与率の閾値を 0.8 とする
cumsum_variance_ratio_threshold = 0.8
# まずは全く圧縮しないPCAを行う
pca_full = PCA(
    n_components=data_classify_train_X_normed.shape[1], random_state=14
).fit(data_classify_train_X_normed)
# 累積寄与率のグラフを描く
pd.DataFrame({
    'dimension': range(1, data_classify_train_X_normed.shape[1] + 1),
    'cumsum_variance_ratio': np.cumsum(pca_full.explained_variance_ratio_),
}).plot(kind='line', x='dimension', y='cumsum_variance_ratio')
plt.ylabel('cumsum variance ratio')
plt.title('dimension vs cumsum variance ratio')
plt.axhline(cumsum_variance_ratio_threshold, color='red')
plt.legend().remove()

In [ ]:
# 圧縮後の次元数の計算
decomposed_dimension = range(
    1, data_classify_train_X_normed.shape[1] + 1
)[
    np.min(np.where(np.cumsum(
        pca_full.explained_variance_ratio_
    ) > cumsum_variance_ratio_threshold))
]
decomposed_dimension

In [ ]:
# 算出した次元数へ圧縮
pca_decompose = PCA(
    n_components=decomposed_dimension, random_state=14
).fit(data_classify_train_X_normed)
# 学習データを圧縮
data_classify_train_X_decomposed = pca_decompose.transform(
    data_classify_train_X_normed
)
# 検証データを圧縮
data_classify_test_X_decomposed = pca_decompose.transform(
    data_classify_test_X_normed
)

In [ ]:
data_classify_train_X_decomposed.shape

In [ ]:
pd.DataFrame(data_classify_train_X_decomposed).head()

## Support Vector Machine

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

# Cross Validation で探索するHyper Parameter
# 目的変数がimbalanceの場合は、`class_weight` として None と 'balanced' を探索する
hparams_classify_svc = {
    'C': expon(scale=1),
    'class_weight': [None, 'balanced'],
}
# Cross Validation
# マルチコが発生する可能性があるため、ペナルティをL2に設定
# Cross Validation の評価指標としてはF1値を採用
# 特徴量は、特徴量エンジニアリングを行い、正規化したものを用いる
svc_classify = LinearSVC(penalty='l2', max_iter=1000, random_state=14)
skf_classify_svc = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_classify_svc = RandomizedSearchCV(
    svc_classify, hparams_classify_svc, cv=skf_classify_svc, scoring='f1',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_classify_train_X_normed, data_classify_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_classify_svc.best_params_.items()
])

In [ ]:
# 学習データで作成した最善モデルの、予測結果のF1値
hpsearch_classify_svc.best_score_

In [ ]:
# 偏回帰係数の可視化
pd.DataFrame({
    'feature': ['(intercept)'] + list(data_classify_train_X.columns),
    'coef': list(
        hpsearch_classify_svc.best_estimator_.intercept_
    ) + list(
        hpsearch_classify_svc.best_estimator_.coef_[0]
    ),
}).plot(kind='barh', x='feature', y='coef')
plt.xlabel('value of coefficient')
plt.title('coefficient')
plt.legend().remove()

In [ ]:
from sklearn.metrics import classification_report

# 検証データで予測する
pred_classify_svc_test = hpsearch_classify_svc.best_estimator_.predict(
    data_classify_test_X_normed
)
# 検証データの予測結果を評価する
pd.DataFrame(classification_report(
    data_classify_test_y, pred_classify_svc_test, output_dict=True,
    target_names=['smorker_is_no', 'smorker_is_yes']
)).T

In [ ]:
from sklearn.metrics import confusion_matrix

# 検証データの正解・不正解を可視化する
pd.DataFrame(
    confusion_matrix(data_classify_test_y, pred_classify_svc_test).T,
    index=['predicted_smorker_is_no', 'predicted_smorker_is_yes'],
    columns=['smorker_is_no', 'smorker_is_yes']
).loc[
    ['predicted_smorker_is_yes', 'predicted_smorker_is_no'],
    ['smorker_is_yes', 'smorker_is_no']
]

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Cross Validation で探索するHyper Parameter
# 目的変数がimbalanceの場合は、`class_weight` として None と 'balanced' を探索する
hparams_classify_rfc = {
    'n_estimators': randint(low=10, high=100),
    'max_depth': randint(low=2, high=10),
    'class_weight': [None, 'balanced'],
}
# Cross Validation
# マルチコが発生する可能性があるため、ペナルティをL2に設定
# Cross Validation の評価指標としてはF1値を採用
# 特徴量は、特徴量エンジニアリングは行ったが、正規化していないものを用いる
rfc_classify = RandomForestClassifier(random_state=14)
skf_classify_rfc = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_classify_rfc = RandomizedSearchCV(
    rfc_classify, hparams_classify_rfc, cv=skf_classify_rfc, scoring='f1',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_classify_train_X, data_classify_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_classify_rfc.best_params_.items()
])

In [ ]:
# 学習データで作成した最善モデルの、予測結果のF1値
hpsearch_classify_rfc.best_score_

In [ ]:
# feature importance（特徴量の重要度）の可視化
pd.DataFrame({
    'feature': data_classify_train_X.columns,
    'importance': hpsearch_classify_rfc.best_estimator_.feature_importances_,
}).plot(kind='barh', x='feature', y='importance')
plt.xlabel('importance')
plt.title('importance of features')
plt.legend().remove()

In [ ]:
# 検証データで予測する
pred_classify_rfc_test = hpsearch_classify_rfc.best_estimator_.predict(
    data_classify_test_X
)
# 検証データの予測結果を評価する
pd.DataFrame(classification_report(
    data_classify_test_y, pred_classify_rfc_test, output_dict=True,
    target_names=['smorker_is_no', 'smorker_is_yes']
)).T

In [ ]:
# 検証データの正解・不正解を可視化する
pd.DataFrame(
    confusion_matrix(data_classify_test_y, pred_classify_rfc_test).T,
    index=['predicted_smorker_is_no', 'predicted_smorker_is_yes'],
    columns=['smorker_is_no', 'smorker_is_yes']
).loc[
    ['predicted_smorker_is_yes', 'predicted_smorker_is_no'],
    ['smorker_is_yes', 'smorker_is_no']
]

# モデル作成と評価（回帰）

* 目的変数として `charges` を設定する
    * 各種データから、その人の医療費を予測する

## 目的変数・特徴量の分離

In [ ]:
# 目的変数・特徴量の分離
data_regress_y = data['charges']
data_regress_X = data.drop('charges', axis='columns')
# データ数の確認
data_regress_X.shape, data_regress_y.shape

## 学習用・検証用データの分離

In [ ]:
# 学習用・検証用データの分離
# 回帰なのでimbalanceという概念は存在せず、stratifyは出来ない
(
    data_regress_train_X, data_regress_test_X,
    data_regress_train_y, data_regress_test_y
) = train_test_split(
    data_regress_X, data_regress_y,
    test_size=0.3, random_state=14
)

## Support Vector Machine

In [ ]:
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVR

# Cross Validation で探索するHyper Parameter
hparams_regress_svr = {
    'C': expon(scale=1),
}
# Cross Validation
# Cross Validation の評価指標としては、`charges` がべき分布なので、MSLE（平均２乗対数誤差）を採用
# 線形 Shallow モデルで回帰を行うが、目的変数に大きな値が含まれる場合、
# 偏回帰係数がペナルティで抑え込まれてしまうため、非常に悪い結果になる。
# よって、今回は特徴量の正規化をわざと行わない。
svr_regress = LinearSVR(max_iter=1000, random_state=14)
kf_regress_svr = KFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_regress_svr = RandomizedSearchCV(
    svr_regress, hparams_regress_svr, cv=kf_regress_svr,
    scoring='neg_mean_squared_log_error',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_regress_train_X, data_regress_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_regress_svr.best_params_.items()
])

In [ ]:
# 偏回帰係数の可視化
pd.DataFrame({
    'feature': ['(intercept)'] + list(data_regress_train_X.columns),
    'coef': list(
        hpsearch_regress_svr.best_estimator_.intercept_
    ) + list(
        hpsearch_regress_svr.best_estimator_.coef_
    ),
}).plot(kind='barh', x='feature', y='coef')
plt.xlabel('coefficient')
plt.title('value of coefficient')
plt.legend().remove()

In [ ]:
# 検証データで予測する
pred_regress_svr_test = hpsearch_regress_svr.best_estimator_.predict(
    data_regress_test_X
)
# 検証データの予測結果を可視化する
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
pd.DataFrame({
    'true': data_regress_test_y,
    'predicted': pred_regress_svr_test
}).plot(kind='scatter', x='true', y='predicted', ax=ax, color=sns.color_palette('tab10')[0])
maxval = max(np.max(data_regress_test_y), np.max(pred_regress_svr_test))
ax.plot((0, maxval), (0, maxval), color='red')
plt.xlim((0, 1.03 * maxval))
plt.ylim((0, 1.03 * maxval))
plt.title('prediction of `charges` (Support Vector Machine)')

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Cross Validation で探索するHyper Parameter
hparams_regress_rfr = {
    'n_estimators': randint(low=10, high=200),
    'max_depth': randint(low=2, high=10),
}
# Cross Validation
# Cross Validation の評価指標としては、`charges` がべき分布なので、MSLE（平均２乗対数誤差）を採用
rfr_regress = RandomForestRegressor(random_state=14)
kf_regress_rfr = KFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_regress_rfr = RandomizedSearchCV(
    rfr_regress, hparams_regress_rfr, cv=kf_regress_rfr,
    scoring='neg_mean_squared_log_error',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_regress_train_X, data_regress_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_regress_rfr.best_params_.items()
])

In [ ]:
# feature importance（特徴量の重要度）の可視化
pd.DataFrame({
    'feature': data_regress_train_X.columns,
    'importance': hpsearch_regress_rfr.best_estimator_.feature_importances_,
}).plot(kind='barh', x='feature', y='importance')
plt.xlabel('importance')
plt.title('importance of features')
plt.legend().remove()

In [ ]:
# 検証データで予測する
pred_regress_rfr_test = hpsearch_regress_rfr.best_estimator_.predict(
    data_regress_test_X
)
# 検証データの予測結果を可視化する
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
pd.DataFrame({
    'true': data_regress_test_y,
    'predicted': pred_regress_rfr_test
}).plot(kind='scatter', x='true', y='predicted', ax=ax, color=sns.color_palette('tab10')[0])
maxval = max(np.max(data_regress_test_y), np.max(pred_regress_svr_test))
ax.plot((0, maxval), (0, maxval), color='red')
plt.xlim((0, 1.03 * maxval))
plt.ylim((0, 1.03 * maxval))
plt.title('prediction of `charges` (Random Forest)')

In [ ]:
# 誤差の可視化
(pred_regress_rfr_test - data_regress_test_y).plot(kind='hist', bins=20, logy=True)
plt.xlabel('error')
plt.title('distribution of regresssion error')

In [ ]:
from sklearn.metrics import mean_squared_log_error

# 誤差の評価値
mean_squared_log_error(data_regress_test_y, pred_regress_rfr_test)

# モデルの Stacking

* やはり回帰問題は難しい
    * Support Vector Machine では、予測結果が３つの部分に分かれており、それぞれに bias を掛ける（ゲタを履かせる）ことが必要そう
        * ちょうど `charges` を３つにクラスタリングした結果のように
    * Random Forest では、真ん中のクラスタに相当するデータで誤差が大きい
* 回帰は難しくても、分類は出来るのではないか
    * `charges` のクラスタを予測する
* 本当に `charges` の詳細な値を予測することが必要だったのか
    * 例えば `charges` を予測する目的が「高額の医療費を払っている人に新しい保険商品を宣伝したい」というものであった場合
    * 正確な医療費を予測することは必要なく、「高額の医療費を払っている人」（cluster2に相当）を予測するだけで良い
    * **より簡単で精度の高い手法を用いて、効果的にビジネス目的（KPI）を実現する**
* それでもやはり `charges` の値を予測することが必要だった
    * 分類でクラスタを精度良く予測できた、とする
    * その値を特徴量に加えることで、`charges` の値を精度良く予測することが出来ないだろうか
    * ２段階の予測を行う（分類し、そして回帰する）
* Ensemble と Stacking
    * 複数のモデルを組み合わせる手法は、大きく２種類存在する
    * Ensemble
        * 複数のモデルの予測結果を寄せ集めて、その結果から最終的な値を予測する
    * Stacking
        * あるモデルの予測結果を特徴量に加え、そして次のモデルの予測を行う
    * この場合は Stacking に相当
* 分析方針
    * `charges` を1次元 k-Means したものを用い、`charges` をクラスタに変換したものを目的変数とし、分類を行う
        * 精度がわずかに良かった Random Forest を用いる
    * その結果を特徴量に加え、`charges` を予測する回帰を行う
        * こちらも Random Forest を用いる
        * Support Vector Machine を用いても、偏回帰係数にかかるペナルティにより、bias に十分な値にならない
        * Support Vecotr Machine を用いるなら、予測したクラスタ毎に分けて学習し、予測するべき

## Stacking １段目：クラスタの分類

In [ ]:
# `charges`（回帰の目的変数）をクラスタ番号に変換
data_stack_train_y = [
    int(km_cluster_map[x][-1]) for x in km_charges.predict(
        ss_kmeans.transform(data_regress_train_y.to_numpy().reshape(-1, 1))
    )
]
data_stack_test_y = [
    int(km_cluster_map[x][-1]) for x in km_charges.predict(
        ss_kmeans.transform(data_regress_test_y.to_numpy().reshape(-1, 1))
    )
]

In [ ]:
# クラスタに属するデータの個数の分布（学習データ）
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data_stack_train_y)).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# クラスタに属するデータの個数の分布（評価データ）
pd.DataFrame([
    {'key': k, 'value': v} for k, v in dict(Counter(data_stack_test_y)).items()
]).sort_values('key').reset_index(drop=True)

In [ ]:
# １段目の学習

# Cross Validation で探索するHyper Parameter
hparams_stack_rfc = {
    'n_estimators': randint(low=10, high=100),
    'max_depth': randint(low=2, high=10),
    'class_weight': [None, 'balanced'],
}
# Cross Validation
# `smorker`の分類を行った時は２値分類だったが、
# 今回は３クラスの分類（multiclass classification）であるため、
# Cross Validation の評価値として F1 macro を用いる
rfc_stack = RandomForestClassifier(random_state=14)
skf_stack_rfc = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_stack_rfc = RandomizedSearchCV(
    rfc_stack, hparams_stack_rfc, cv=skf_stack_rfc, scoring='f1_macro',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_regress_train_X, data_stack_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_stack_rfc.best_params_.items()
])

In [ ]:
# 学習データで作成した最善モデルの、予測結果のF1値
hpsearch_stack_rfc.best_score_

In [ ]:
# feature importance（特徴量の重要度）の可視化
pd.DataFrame({
    'feature': data_regress_train_X.columns,
    'importance': hpsearch_stack_rfc.best_estimator_.feature_importances_,
}).plot(kind='barh', x='feature', y='importance')
plt.xlabel('importance')
plt.title('importance of features')
plt.legend().remove()

In [ ]:
# 検証データで予測する
pred_stack_rfc_test = hpsearch_stack_rfc.best_estimator_.predict(
    data_regress_test_X
)
# 検証データの予測結果を評価する
pd.DataFrame(classification_report(
    data_stack_test_y, pred_stack_rfc_test, output_dict=True,
    target_names=['cluster0', 'cluster1', 'cluster2']
)).T

In [ ]:
# 検証データの正解・不正解を可視化する
pd.DataFrame(
    confusion_matrix(data_stack_test_y, pred_stack_rfc_test).T,
    index=['predicted_cluster0', 'predicted_cluster1', 'predicted_cluster2'],
    columns=['cluster0', 'cluster1', 'cluster2']
).loc[
    ['predicted_cluster0', 'predicted_cluster1', 'predicted_cluster2'],
    ['cluster0', 'cluster1', 'cluster2']
]

## Stacking ２段目：`charges` の回帰

In [ ]:
# １段目の結果を特徴量に入れ込む
data_stack_train_X = data_regress_train_X.assign(
    charge_cluster=hpsearch_stack_rfc.best_estimator_.predict(
        data_regress_train_X
    )
)
# クラスタ番号をダミー変数化
data_stack_train_X = pd.concat([
    data_stack_train_X,
    pd.get_dummies(data_stack_train_X['charge_cluster'], prefix='charge_cluster', drop_first=True),
], axis='columns')
# `charge_cluster`を除去
data_stack_train_X.drop('charge_cluster', axis='columns', inplace=True)
# 先頭５行を表示
data_stack_train_X.head(n=5)

In [ ]:
# ２段目の学習

# Cross Validation で探索するHyper Parameter
hparams_stack_rfr = {
    'n_estimators': randint(low=10, high=200),
    'max_depth': randint(low=2, high=10),
}
# Cross Validation
rfr_stack = RandomForestRegressor(random_state=14)
kf_stack_rfr = KFold(n_splits=5, shuffle=True, random_state=14)
hpsearch_stack_rfr = RandomizedSearchCV(
    rfr_stack, hparams_stack_rfr, cv=kf_stack_rfr,
    scoring='neg_mean_squared_log_error',
    n_iter=10, n_jobs=-1, refit=True, random_state=14
).fit(data_stack_train_X, data_regress_train_y)

In [ ]:
# Cross Validation の結果、最善とされた Hyper Parameter
pd.DataFrame([
    {'Hyper Parameter': k, 'value': v} for k, v in hpsearch_stack_rfr.best_params_.items()
])

In [ ]:
# feature importance（特徴量の重要度）の可視化
pd.DataFrame({
    'feature': data_stack_train_X.columns,
    'importance': hpsearch_stack_rfr.best_estimator_.feature_importances_,
}).plot(kind='barh', x='feature', y='importance')
plt.xlabel('importance')
plt.title('importance of features')
plt.legend().remove()

In [ ]:
# 検証データで１段目の結果の入れ込みから評価まで一気にやる

# １段目の結果を特徴量に入れ込む
data_stack_test_X = data_regress_test_X.assign(
    charge_cluster=hpsearch_stack_rfc.best_estimator_.predict(
        data_regress_test_X
    )
)
# クラスタ番号をダミー変数化
data_stack_test_X = pd.concat([
    data_stack_test_X,
    pd.get_dummies(data_stack_test_X['charge_cluster'], prefix='charge_cluster', drop_first=True),
], axis='columns')
# `charge_cluster`を除去
data_stack_test_X.drop('charge_cluster', axis='columns', inplace=True)
# `charges`の予測
pred_stack_rfr_test = hpsearch_stack_rfr.best_estimator_.predict(
    data_stack_test_X
)
# 検証データの予測結果を可視化する
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
pd.DataFrame({
    'true': data_regress_test_y,
    'predicted': pred_stack_rfr_test
}).plot(kind='scatter', x='true', y='predicted', ax=ax, color=sns.color_palette('tab10')[0])
maxval = max(np.max(data_regress_test_y), np.max(pred_stack_rfr_test))
ax.plot((0, maxval), (0, maxval), color='red')
plt.xlim((0, 1.03 * maxval))
plt.ylim((0, 1.03 * maxval))
plt.title('prediction of `charges` (Stacking)')

In [ ]:
# 誤差の可視化
(pred_stack_rfr_test - data_regress_test_y).plot(kind='hist', bins=20, logy=True)
plt.xlabel('error')
plt.title('distribution of regresssion error')

In [ ]:
# 誤差の評価値
mean_squared_log_error(data_regress_test_y, pred_stack_rfr_test)